In [42]:
import os,re
import requests as rq
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import spotipy
import lyricsgenius as genius
import urllib3
import unidecode # 3rd party
import string
from bs4 import BeautifulSoup

In [16]:
#rapgenius information
rap_genius_id = "_17NJOxR9_RGp1XTQ1Vl3O--bxYh-cSzcCwASKlr-Xlyj3vh2_BsdedBZLX6xgdt"
client_secret = "2wlydcjFiqOlSI5Zi65OeizvaWRx16xkkjIBfPkqZ9t1aFNNYTrjza8ny6xYlXdJQSP_vMzj6e20O7ORtjIIcQ"
client_access_token = "Uzv_tj6uOXE9_1eX2wCrOGi3j4JtmYcm1kVF-UmPcC5gJh1MQZKJLph4aFp9frFx"

In [158]:
# make rappers from data folder
def get_rappers():
    
    def validate(name):
        # checks if the name has a
        illegal_chars = "|?:^*\""

        for c in illegal_chars:
            if c == '|':
                if name.find(c) > 0:
                    name = name.split("|")[1]
            if c == "\"":
                if name.find(c) > 0:
                    name = name.replace(r'"','',2)
            else: 
                name = name.replace(c,'')
        return name
    
    with open("data/rappers.txt", "r", encoding='utf-8') as f:
        rappers = f.read()
        
    return [validate(rapper).strip() for rapper in rappers.split("\n")]
rappers = get_rappers() 

In [209]:
"Talk Up (feat. Jay-Z)".partition('(')[0].strip()

'Talk Up'

In [269]:
def song_lyrics(artist, track_title):
    track_title = track_title.replace("&", "and").partition("(")[0].strip()
    def _process(text):
        # formats the url
        puncs = set(string.punctuation) - set("-")
        text = [''.join([c for c in s if c not in puncs]).lower() 
                  for s in unidecode.unidecode(text).split()]
        return text
        
    artist = "-".join(_process(artist)).capitalize()
    track_title = "-".join(_process(track_title))
    
    URL = fr"https://genius.com/{'-'.join([artist, track_title])}-lyrics"
    
    # request page to get markup
    page = rq.get(URL)
    html = BeautifulSoup(page.text, "html.parser")
    lyrics = html.find("div", class_="lyrics").get_text()

    lyrics = " ".join([line for line in lyrics.strip("\n").split("\n") 
                       if not line.startswith('[') and line != ""])
    
    return " ".join(_process(lyrics))

In [ ]:
song_lyrics('drakE', 'in mY FEElings')

In [186]:
def retrieve_albums(artist):
    with open(f'data/albums/{artist}.txt', 'r', encoding='utf8') as f:
        info = eval(f.read())
    return info

In [ ]:
def get_all_lyrics(artist):
    print(f"getting albums for {artist}")
    info = retrieve_albums(artist)
    
    albums = []
    for album in info:
        album_info = {}
        print("  "+album['name'])
        album_info['name'] = album['name']
        album_info['date'] = album['release_date']
        album_info['tracks'] = list()
        for track in album['tracks']:
            try:
                lyrics = song_lyrics(artist, track['track_title'])
            except:
                continue
            print(f"Song: {track['track_title']}")
            album_info['tracks'].append({'name':track['track_title'],
             'lyrics': lyrics})
        albums.append(album_info)
    return albums

In [ ]:
errors = []
for r in rappers:
    print(f"done: {r}")
    with open(f"data/lyrics/{r}.txt", 'w', encoding='utf8') as f:
        try:
            f.write(str(get_all_lyrics(r)))
        except SyntaxError:
            errors.append(r)
            continue

done: 03 Greedo
getting albums for 03 Greedo
  The Wolf of Grape Street
Song: Drippin'
Song: Ballin' (feat. Ketchy The Great)
Song: If I Wasn't Rappin'
Song: Bacc to Bacc (feat. Yhung T.O.)
Song: Baytoven
Song: Beat That Thang Down (feat. PNB Rock)
Song: Run For Yo Life
Song: Look At Me Now
Song: Voodoo
Song: 100 Bands (feat. OMB Peezy)
Song: Paranoid Pt. 03 (feat. Bambino)
Song: Vulture (feat. Ketchy The Great)
Song: Chase (feat. Ketchy The Great & Doley Bernays)
Song: White Owl (feat. Ketchy The Great)
Song: Safety (feat. Ralfy The Plug)
Song: For My Dawgs
Song: Pop It
Song: Substance
Song: Rude
Song: Zoning
Song: Never Bend
  Purple Summer 2: Son Don't Shine
Song: Paranoid
Song: Baby Sittin
  God Level
Song: Blower (feat. AD)
Song: Finally
Song: Fall Off
Song: Basehead
Song: Prayer For My Lost
Song: Buckhead (feat. KEY, Runway Richy, ManMan Savage & Mal & Quill)
Song: In My Feelings
Song: Praying to God
Song: Gun Bucc
Song: Floating
Song: Dibiase
Song: Conscience
Song: Fortnite (Rem